In [1]:
import string
import pandas as pd
import numpy as np
import re

In [2]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [3]:
#Opening text file found online with correct Journal Titles in it

lines = [] 
with open ('J_Medline.txt', 'rt') as in_file:
    for line in in_file:
        lines.append(line)


In [4]:
journallines = []
journalabbr = []


In [5]:
# Code to strip lines with journal title and lines with medical abbriviations 

for x in lines:
    if x.startswith('JournalTitle: '):
        x = x.rstrip()
        x = x.replace('JournalTitle: ', '')
        journallines.append(x)
    if x.startswith('MedAbbr: '):
        x = x.rstrip()
        x = x.replace('MedAbbr: ', '')
        journalabbr.append(x)
        

In [6]:
#It looks like it worked!

len(journallines)


30346

In [7]:
#It looks like it worked again!

len(journalabbr)


30346

In [8]:
#######################################################
#### Getting the Database #############################
#######################################################

df = pd.read_csv('Welcome.csv', encoding = "latin1")

In [9]:
# Setting cleaned column so I know what data has been affected
df['Cleaned']=0
df['NewTitle']=''
dfclean = df
# Taking out the number 1 and replacing it with 'one'
dfclean['Journal title']=(dfclean['Journal title'].replace('[1]','one', regex=True ))
dfnew = dfclean

In [10]:
# It is basically the same algorithm as below with a few tweaks
# I found it works better to search for abbreviations first and then after search for journal titles
# This algorithm compares journal abbrviations with journal abbreviations found on the web, 
# if there is a 90% or more match it changes it the journal title to the coresponding title

#Iterate through rows that were not cleaned, looking for abbreviations
for index, row in dfclean.iterrows():
    q = 0
    lastq=0
    print('------------------------------------------------\nID: ' + str(index))
    
    # Count is used to find the correct line in the journal title list 'journallines'
    count = 0
    
    # Search for abbr on the ones that have not been cleaned (aka Cleaned == 0)
    
    if row['Cleaned']==0:
    
    # Make sure it is a string    
        if type(row['Journal title']) == str:

            # Iterate through abbreviations from the website
            for k in journalabbr:
            
                # Make both titles/abbr. being compared lowercased. Also, remove 'the'
                dbname = row['Journal title'].lower()
                webname = k.lower()
                dbname=dbname.replace('the ','')
                webname=webname.replace('the ','')
            
                q=fuzz.ratio(webname, dbname) #Check ratio

        
        # Change name and switch to next row of dirty dataframe if there is a perfect match
                if q == 100: 
                    dfnew.at[index, 'NewTitle'] = journallines[count]
                    dfnew.at[index, 'Cleaned'] = q
                    print(' Perfect Abbr. Match! ' + journallines[count] + " |||| Old title : " + k)
                    break
                
#                if q >= 80:
#                    z=fuzz.partial_ratio(webname, dbname)
#                    if z >= 80: #Check partial ratio
#                        q=q+5
                  
        
        # Check if the ratio is good enough to change it
                if q >= 85 and q >= lastq:
                    dfnew.at[index, 'Cleaned'] = q
                    lastq = q
                    print(' Score:' + str(lastq) + '\nOG title: '+ row['Journal title'] + ' ||||| Possible Title: ' + journallines[count])
                    dfnew.at[index, 'NewTitle'] = journallines[count]
                
                # Count is used to find the correct line in the journal title list 'journallines', this is needed
                # because we are itterating through the abbreviation list
                count = count+1
                
    else:
        print(' Already Cleaned ')

                    
                
#Save it after compile        
dfnew.to_csv('CleanedAbbr.csv')

------------------------------------------------
ID: 0
------------------------------------------------
ID: 1
 Score:90
OG title: Biomacromolecules ||||| Possible Title: Macromolecules
 Perfect Abbr. Match! Biomacromolecules |||| Old title : Biomacromolecules
------------------------------------------------
ID: 2
 Perfect Abbr. Match! Journal of medicinal chemistry |||| Old title : J Med Chem
------------------------------------------------
ID: 3
 Perfect Abbr. Match! Journal of medicinal chemistry |||| Old title : J Med Chem
------------------------------------------------
ID: 4
 Perfect Abbr. Match! The Journal of organic chemistry |||| Old title : J Org Chem
------------------------------------------------
ID: 5
------------------------------------------------
ID: 6
------------------------------------------------
ID: 7
 Perfect Abbr. Match! Molecular pharmaceutics |||| Old title : Mol Pharm
------------------------------------------------
ID: 8
-------------------------------------

------------------------------------------------
ID: 110
------------------------------------------------
ID: 111
------------------------------------------------
ID: 112
------------------------------------------------
ID: 113
------------------------------------------------
ID: 114
------------------------------------------------
ID: 115
------------------------------------------------
ID: 116
------------------------------------------------
ID: 117
 Score:86
OG title: J Biol Chem. ||||| Possible Title: Journal of biochemistry
 Score:96
OG title: J Biol Chem. ||||| Possible Title: The Journal of biological chemistry
------------------------------------------------
ID: 118
 Score:86
OG title: J Biol Chem. ||||| Possible Title: Journal of biochemistry
 Score:96
OG title: J Biol Chem. ||||| Possible Title: The Journal of biological chemistry
------------------------------------------------
ID: 119
 Score:86
OG title: J Biol Chem. ||||| Possible Title: Journal of biochemistry
 Score:96
O

------------------------------------------------
ID: 203
 Score:88
OG title: Genome Biology ||||| Possible Title: Genome biology
------------------------------------------------
ID: 204
 Score:88
OG title: Genome Biology ||||| Possible Title: Genome biology
------------------------------------------------
ID: 205
------------------------------------------------
ID: 206
------------------------------------------------
ID: 207
------------------------------------------------
ID: 208
------------------------------------------------
ID: 209
------------------------------------------------
ID: 210
------------------------------------------------
ID: 211
------------------------------------------------
ID: 212
------------------------------------------------
ID: 213
------------------------------------------------
ID: 214
------------------------------------------------
ID: 215
------------------------------------------------
ID: 216
------------------------------------------------
ID: 217
-

 Score:91
OG title: Trials ||||| Possible Title: Trial (Boston, Mass.)
 Perfect Abbr. Match! Trials |||| Old title : Trials
------------------------------------------------
ID: 277
 Score:91
OG title: Trials ||||| Possible Title: Trial (Boston, Mass.)
 Perfect Abbr. Match! Trials |||| Old title : Trials
------------------------------------------------
ID: 278
 Score:89
OG title: BJOPHTHALMOL ||||| Possible Title: The British journal of ophthalmology
 Score:92
OG title: BJOPHTHALMOL ||||| Possible Title: Journal of ophthalmology
------------------------------------------------
ID: 279
 Perfect Abbr. Match! BMJ (Clinical research ed.) |||| Old title : BMJ
------------------------------------------------
ID: 280
 Perfect Abbr. Match! BMJ (Clinical research ed.) |||| Old title : BMJ
------------------------------------------------
ID: 281
 Perfect Abbr. Match! BMJ (Clinical research ed.) |||| Old title : BMJ
------------------------------------------------
ID: 282
 Perfect Abbr. Match! BMJ

------------------------------------------------
ID: 369
 Score:88
OG title: Public Health Nutrition ||||| Possible Title: Public health nutrition
------------------------------------------------
ID: 370
------------------------------------------------
ID: 371
 Perfect Abbr. Match! Urban history |||| Old title : Urban History
------------------------------------------------
ID: 372
------------------------------------------------
ID: 373
------------------------------------------------
ID: 374
------------------------------------------------
ID: 375
 Score:87
OG title: Stem Cell Reports ||||| Possible Title: Stem cell research
 Perfect Abbr. Match! Stem cell reports |||| Old title : Stem Cell Reports
------------------------------------------------
ID: 376
------------------------------------------------
ID: 377
------------------------------------------------
ID: 378
------------------------------------------------
ID: 379
------------------------------------------------
ID: 380
-----

...

------------------------------------------------
ID: 1760
------------------------------------------------
ID: 1761
------------------------------------------------
ID: 1762
------------------------------------------------
ID: 1763
------------------------------------------------
ID: 1764
------------------------------------------------
ID: 1765
------------------------------------------------
ID: 1766
------------------------------------------------
ID: 1767
------------------------------------------------
ID: 1768
------------------------------------------------
ID: 1769
------------------------------------------------
ID: 1770
------------------------------------------------
ID: 1771
------------------------------------------------
ID: 1772
------------------------------------------------
ID: 1773
 Perfect Abbr. Match! Metabolomics : Official journal of the Metabolomic Society |||| Old title : Metabolomics
------------------------------------------------
ID: 1774
-------------------

------------------------------------------------
ID: 1834
------------------------------------------------
ID: 1835
------------------------------------------------
ID: 1836
 Perfect Abbr. Match! Pediatrics |||| Old title : Pediatrics
------------------------------------------------
ID: 1837
 Perfect Abbr. Match! Virology |||| Old title : Virology
------------------------------------------------
ID: 1838
 Score:86
OG title: Biology Open ||||| Possible Title: Biology open
------------------------------------------------
ID: 1839
------------------------------------------------
ID: 1840
 Perfect Abbr. Match! Biology open |||| Old title : Biol Open
------------------------------------------------
ID: 1841
 Score:86
OG title: Biology Open ||||| Possible Title: Biology open
------------------------------------------------
ID: 1842
 Perfect Abbr. Match! Development (Cambridge, England) |||| Old title : Development
------------------------------------------------
ID: 1843
--------------------

------------------------------------------------
ID: 1933
 Score:86
OG title: FEMS Microbiology Letters ||||| Possible Title: FEMS microbiology letters
------------------------------------------------
ID: 1934
 Score:86
OG title: FEMS Microbiology Letters ||||| Possible Title: FEMS microbiology letters
------------------------------------------------
ID: 1935
 Score:89
OG title: Genes Brain Behaviour ||||| Possible Title: Genes, brain, and behavior
------------------------------------------------
ID: 1936
------------------------------------------------
ID: 1937
 Perfect Abbr. Match! Glia |||| Old title : Glia
------------------------------------------------
ID: 1938
 Perfect Abbr. Match! Glia |||| Old title : Glia
------------------------------------------------
ID: 1939
------------------------------------------------
ID: 1940
 Perfect Abbr. Match! Hepatology (Baltimore, Md.) |||| Old title : Hepatology
------------------------------------------------
ID: 1941
 Perfect Abbr. Match! H

 Score:97
OG title: Dev World Bioeth. ||||| Possible Title: Developing world bioethics
------------------------------------------------
ID: 2036
 Score:97
OG title: Dev World Bioeth. ||||| Possible Title: Developing world bioethics
------------------------------------------------
ID: 2037
 Score:97
OG title: Dev. World Bioeth ||||| Possible Title: Developing world bioethics
------------------------------------------------
ID: 2038
 Score:97
OG title: Dev. World Bioeth ||||| Possible Title: Developing world bioethics
------------------------------------------------
ID: 2039
 Perfect Abbr. Match! Epilepsia |||| Old title : Epilepsia
------------------------------------------------
ID: 2040
 Perfect Abbr. Match! Epilepsia |||| Old title : Epilepsia
------------------------------------------------
ID: 2041
 Perfect Abbr. Match! European journal of immunology |||| Old title : Eur J Immunol
------------------------------------------------
ID: 2042
 Score:97
OG title: Influenza Other Respi Vi

 Perfect Abbr. Match! The Pediatric infectious disease journal |||| Old title : Pediatr Infect Dis J
------------------------------------------------
ID: 2125
 Perfect Abbr. Match! AIDS (London, England) |||| Old title : AIDS
------------------------------------------------
ID: 2126


In [69]:
dftitle = pd.read_csv('CleanedAbbr.csv', encoding = "latin1")

In [70]:
dfnew = dftitle

In [71]:
dfnew[dfnew['Cleaned']==0].count()

Unnamed: 0                                              1484
PMID/PMCID                                              1340
Publisher                                               1484
Journal title                                           1483
Article title                                           1484
COST (Â£) charged to Wellcome (inc VAT when charged)    1484
Cleaned                                                 1484
dtype: int64

In [100]:
#######################################################################################
# Algorithm that compares the journal titles within the database with the list of real journal titles found on the website.
# It uses fuzzywuzzy to compare the strings and chooses the best comparison, if no good comparisons are 
# found then nothing happens. There is a cleaned column, 0 represents the title hasnt been changed
# the number will be changed to the ratio of the match (100 for perfect match, 80+ for good match)
########################################################################################


#Iterate through rows of dirty database
for index, row in dfnew.iterrows():
    q = 0
    lastq=0
    print('------------------------------------------------\nID: ' + str(index))
    print(row['Journal title'])
    
    if row['Cleaned']<=95:
    # Make sure it is a string        
        if type(row['Journal title']) == str:

            # Iterate through titles from the website
            for k in journallines:

                # Make both titles being compared lowercased. Also, remove 'the'
                dbname = row['Journal title'].lower()
                webname = k.lower()
                
                # Remove the, of, a and : 
                dbname=dbname.replace('the ','')
                webname=webname.replace('the ','')
                dbname=dbname.replace(' : ','')
                webname=webname.replace(' : ','')
                dbname=dbname.replace('a ','')
                webname=webname.replace('a ','')
                dbname=dbname.replace(' of ','')
                webname=webname.replace(' of ','')
                dbname=dbname.replace('jnl','journal')
                webname=webname.replace('jnl','journal')
                dbname=dbname.replace('America','')
                webname=webname.replace('America','')
                
                q=fuzz.ratio(webname, dbname) #Check ratio

            # Change name and switch to next row of dirty dataframe if there is a perfect match
                if q == 100: 
                    dftitle.at[index, 'NewTitle'] = k
                    dftitle.at[index, 'Cleaned'] = q
                    print(' Perfect Match! ' + k)
                    break
                
             
            # If there is a partial ratio, then it adds a bit to the ratio
#                if q >= 75:
#                    z=fuzz.partial_ratio(webname, dbname)
#                    if z >= 95: #Check partial ratio
#                        q=q+10
                    


            # Check if the ratio is good enough to change it
                if q > 85 and q >= lastq:
                    dftitle.at[index, 'Cleaned'] = q
                    lastq = q
                    print(' Score:' + str(lastq) + '\nOG title: '+ row['Journal title'] + ' ||||| Possible Title: ' + k)
                    dftitle.at[index, 'NewTitle'] = k
    else:
        print('This has already been cleaned: ' + str(row['Cleaned']))
                    
                
#Save it after compile        
dftitle.to_csv('Cleaned.csv')

------------------------------------------------
ID: 0
Psychological Medicine
This has already been cleaned: 100
------------------------------------------------
ID: 1
Biomacromolecules
This has already been cleaned: 100
------------------------------------------------
ID: 2
Journal of medicinal chemistry
This has already been cleaned: 100
------------------------------------------------
ID: 3
Journal of medicinal chemistry
This has already been cleaned: 100
------------------------------------------------
ID: 4
The Journal of organic chemistry
This has already been cleaned: 100
------------------------------------------------
ID: 5
Journal of Medicinal Chemistry
This has already been cleaned: 100
------------------------------------------------
ID: 6
Journal of Proteome Research
This has already been cleaned: 100
------------------------------------------------
ID: 7
Molecular pharmaceutics
This has already been cleaned: 100
------------------------------------------------
ID: 8
ACS C

 Score:105
OG title: Clinical and Vaccine Immunology ||||| Possible Title: Clinical and vaccine immunology : CVI
------------------------------------------------
ID: 87
Eukaryotic cell
This has already been cleaned: 100
------------------------------------------------
ID: 88
Eukaryotic cell
This has already been cleaned: 100
------------------------------------------------
ID: 89
Infection and Immunity
This has already been cleaned: 100
------------------------------------------------
ID: 90
Infection and Immunity
This has already been cleaned: 100
------------------------------------------------
ID: 91
Infection and Immunity
This has already been cleaned: 100
------------------------------------------------
ID: 92
Journal of Bacteriology
This has already been cleaned: 100
------------------------------------------------
ID: 93
Journal of Virology
This has already been cleaned: 100
------------------------------------------------
ID: 94
Journal of Virology
This has already been cleaned

 Score:90
OG title: Molecular and Cellular Proteomics ||||| Possible Title: Molecular & cellular proteomics : MCP
------------------------------------------------
ID: 155
The journal of Biological Chemistry
This has already been cleaned: 100
------------------------------------------------
ID: 156
The Journal of Biological Chemistry
This has already been cleaned: 100
------------------------------------------------
ID: 157
The Journal of Biological Chemistry
This has already been cleaned: 100
------------------------------------------------
ID: 158
The Journal of Biological Chemistry
This has already been cleaned: 100
------------------------------------------------
ID: 159
The Journal of Biological Chemistry
This has already been cleaned: 100
------------------------------------------------
ID: 160
The Journal of Biological Chemistry
This has already been cleaned: 100
------------------------------------------------
ID: 161
Jnl Biological Chemistry
This has already been cleaned: 98
--

 Score:93
OG title: Arthritis Research and Therapy ||||| Possible Title: Arthritis research & therapy
------------------------------------------------
ID: 253
BMC Health Services Research
This has already been cleaned: 100
------------------------------------------------
ID: 254
BMC Infectious Diseases
This has already been cleaned: 100
------------------------------------------------
ID: 255
BMC Medical Ethics
This has already been cleaned: 100
------------------------------------------------
ID: 256
BMC Medical Imaging
This has already been cleaned: 100
------------------------------------------------
ID: 257
BMC Medicine
This has already been cleaned: 100
------------------------------------------------
ID: 258
BMC public health
This has already been cleaned: 100
------------------------------------------------
ID: 259
BMC public health
This has already been cleaned: 100
------------------------------------------------
ID: 260
BMC public health
This has already been cleaned: 100
---

------------------------------------------------
ID: 334
European Journal of Health Law
This has already been cleaned: 100
------------------------------------------------
ID: 335
BMJ open
This has already been cleaned: 100
------------------------------------------------
ID: 336
Journal of Neurology, Neurosurgery and Psychiatry
This has already been cleaned: 99
------------------------------------------------
ID: 337
Biophysical Journal
This has already been cleaned: 100
------------------------------------------------
ID: 338
Jnl Biological Chemistry
This has already been cleaned: 98
------------------------------------------------
ID: 339
Journal of Neuroscience
 Score:91
OG title: Journal of Neuroscience ||||| Possible Title: The European journal of neuroscience
 Score:93
OG title: Journal of Neuroscience ||||| Possible Title: Journal of neuroscience methods
 Score:98
OG title: Journal of Neuroscience ||||| Possible Title: Open journal of neuroscience
------------------------------

...

------------------------------------------------
ID: 1826
South Asian History and Culture
This has already been cleaned: 100
------------------------------------------------
ID: 1827
The History of the Family
------------------------------------------------
ID: 1828
Journal of Experimental Nanoscience
This has already been cleaned: 100
------------------------------------------------
ID: 1829
Journal of medicinal chemistry
This has already been cleaned: 100
------------------------------------------------
ID: 1830
Journal of medicinal chemistry
This has already been cleaned: 100
------------------------------------------------
ID: 1831
Journal of medicinal chemistry
This has already been cleaned: 100
------------------------------------------------
ID: 1832
Journal of medicinal chemistry
This has already been cleaned: 100
------------------------------------------------
ID: 1833
Journal of Applied Physiology
This has already been cleaned: 100
-------------------------------------------

------------------------------------------------
ID: 1891
Arthritis and Rheumatism
This has already been cleaned: 100
------------------------------------------------
ID: 1892
Arthritis and Rheumatism
This has already been cleaned: 100
------------------------------------------------
ID: 1893
Bioethics
This has already been cleaned: 100
------------------------------------------------
ID: 1894
Bioethics
This has already been cleaned: 100
------------------------------------------------
ID: 1895
Bioethics
This has already been cleaned: 100
------------------------------------------------
ID: 1896
Biopolymers
This has already been cleaned: 100
------------------------------------------------
ID: 1897
Birth Defects Research Part A: Clinical and Molecular Teratology
This has already been cleaned: 98
------------------------------------------------
ID: 1898
Birth Defects Research Part A: Clinical and Molecular TeratologyÂ  
 Score:105
OG title: Birth Defects Research Part A: Clinical and Mo

------------------------------------------------
ID: 1958
Journal of Evolutionary Biology
This has already been cleaned: 100
------------------------------------------------
ID: 1959
Journal of Neurochemistry
This has already been cleaned: 100
------------------------------------------------
ID: 1960
Journal of Neurochemistry
This has already been cleaned: 100
------------------------------------------------
ID: 1961
Journal of Neurochemistry
This has already been cleaned: 100
------------------------------------------------
ID: 1962
Journal of Neurochemistry 
This has already been cleaned: 98
------------------------------------------------
ID: 1963
Journal of Neuroendocrinology
This has already been cleaned: 100
------------------------------------------------
ID: 1964
Journal of Pathology
This has already been cleaned: 100
------------------------------------------------
ID: 1965
Journal of Pathology
This has already been cleaned: 100
------------------------------------------------

 Score:88
OG title: Tropical Medicine and International Health ||||| Possible Title: Tropical medicine & international health : TM & IH
------------------------------------------------
ID: 2019
Maternal and Child Nutrition
 Score:93
OG title: Maternal and Child Nutrition ||||| Possible Title: Maternal & child nutrition
------------------------------------------------
ID: 2020
Evaluation in Clinical Practice
 Score:100
OG title: Evaluation in Clinical Practice ||||| Possible Title: Journal of evaluation in clinical practice
------------------------------------------------
ID: 2021
FEBS Journal
This has already been cleaned: 100
------------------------------------------------
ID: 2022
JOURNAL OF NEUROCHEMISTRY
This has already been cleaned: 100
------------------------------------------------
ID: 2023
Traffic (Copenhagen, Denmark)
This has already been cleaned: 100
------------------------------------------------
ID: 2024
British Journal of Dermatology
This has already been cleaned: 100

------------------------------------------------
ID: 2084
Journal of Child Psychology and Psychiatry
 Score:86
OG title: Journal of Child Psychology and Psychiatry ||||| Possible Title: Journal of child psychology and psychiatry, and allied disciplines
 Score:90
OG title: Journal of Child Psychology and Psychiatry ||||| Possible Title: Book supplement to the Journal of child psychology and psychiatry
------------------------------------------------
ID: 2085
Journal of Comparative Neurology
This has already been cleaned: 100
------------------------------------------------
ID: 2086
Journal of Evaluation in Clinical Practice
This has already been cleaned: 100
------------------------------------------------
ID: 2087
Journal of Evaluation in Clinical Practice
This has already been cleaned: 100
------------------------------------------------
ID: 2088
Journal of Microscopy
This has already been cleaned: 100
------------------------------------------------
ID: 2089
Journal of Orthopedic Res

In [37]:
fuzz.partial_ratio('Biology Open', 'MicrobiologyOpen')

87

In [104]:

dftitle

,Unnamed: 0,PMID/PMCID,Publisher,Journal title,Article title,Cost,Cleaned,NewTitle
0,0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,0,100,Psychological medicine
1,1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,2381.04,100,NaN
2,2,23043264 PMC3506128,ACS,Journal of medicinal chemistry,"Fumaroylamino-4,5-epoxymorphinans and related ...",642.56,100,NaN
3,3,23438330 PMC3646402,ACS,Journal of medicinal chemistry,Orvinols with mixed kappa/mu opioid receptor a...,669.64,100,NaN
4,4,23438216 PMC3601604,ACS,The Journal of organic chemistry,Regioselective opening of myo-inositol orthoes...,685.88,100,NaN
5,5,PMC3579457,ACS,Journal of Medicinal Chemistry,Comparative Structural and Functional Studies ...,2392.2,100,Journal of medicinal chemistry
6,6,PMC3709265,ACS,Journal of Proteome Research,Mapping Proteolytic Processing in the Secretom...,2367.95,100,Journal of proteome research
7,7,23057412 PMC3495574,ACS,Molecular pharmaceutics,Quantitative silencing of EGFP reporter gene b...,649.33,100,NaN
8,8,PMCID: PMC3780468,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,A Novel Allosteric Inhibitor of the Uridine Di...,1294.59,100,ACS chemical biology
9,9,PMCID: PMC3621575,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,Chemical proteomic analysis reveals the drugab...,1294.78,100,ACS chemical biology


In [105]:
#Change the name cost column to look better
dftitle.rename(columns={'COST (Â£) charged to Wellcome (inc VAT when charged)':'Cost'}, inplace = True) 
dtcleancost = dftitle
dftitle


,Unnamed: 0,PMID/PMCID,Publisher,Journal title,Article title,Cost,Cleaned,NewTitle
0,0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,0,100,Psychological medicine
1,1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,2381.04,100,NaN
2,2,23043264 PMC3506128,ACS,Journal of medicinal chemistry,"Fumaroylamino-4,5-epoxymorphinans and related ...",642.56,100,NaN
3,3,23438330 PMC3646402,ACS,Journal of medicinal chemistry,Orvinols with mixed kappa/mu opioid receptor a...,669.64,100,NaN
4,4,23438216 PMC3601604,ACS,The Journal of organic chemistry,Regioselective opening of myo-inositol orthoes...,685.88,100,NaN
5,5,PMC3579457,ACS,Journal of Medicinal Chemistry,Comparative Structural and Functional Studies ...,2392.2,100,Journal of medicinal chemistry
6,6,PMC3709265,ACS,Journal of Proteome Research,Mapping Proteolytic Processing in the Secretom...,2367.95,100,Journal of proteome research
7,7,23057412 PMC3495574,ACS,Molecular pharmaceutics,Quantitative silencing of EGFP reporter gene b...,649.33,100,NaN
8,8,PMCID: PMC3780468,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,A Novel Allosteric Inhibitor of the Uridine Di...,1294.59,100,ACS chemical biology
9,9,PMCID: PMC3621575,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,Chemical proteomic analysis reveals the drugab...,1294.78,100,ACS chemical biology


In [106]:
# Remove pound symbol and convert to float so we are able to do calculations 

for index, row in dftitle.iterrows():
    
    if type(row['Cost']) == str:
        
        # Remove symbol
        if '£' in row['Cost']:
            dfcleancost.at[index, 'Cost'] = float(row['Cost'].replace("Â£", ""))
            
        # If it's a dollar amount, remove the dollar sign and calculate it in pounds
        if '$' in row ['Cost']:
            new = float(row['Cost'].replace("$", ""))
            # Pound to dollar conversion rate august 2018
            new = round(new * .78, 2)
            dfcleancost.at[index, 'Cost'] = new
        
            
        

In [107]:
# Remove rows with null values as cost
dfcleancost = dfcleancost[pd.notnull(dfcleancost['Cost'])]


# Remove cost outliers, anything less than 50 and greater than 5000
#I should have removed these before the title sorting algorithm,  it would have lowered compile time :(

dfcleancost = dfcleancost.loc[dfcleancost['Cost'] <= 6000]


# dfcleancost = dfcleancost.loc[dfcleancost['Cost'] >= 50]

In [97]:
dfcleancost.to_csv('AllClean.csv')

In [108]:
dfgrouped = dfcleancost.groupby('NewTitle').count()
dfgrouped

,Unnamed: 0,PMID/PMCID,Publisher,Journal title,Article title,Cost,Cleaned
NewTitle,,,,,,,
ACS chemical biology,5,4,5,5,5,5,5
ACS chemical neuroscience,1,1,1,1,1,1,1
AIDS research and therapy,1,1,1,1,1,1,1
"Acta crystallographica. Section D, Biological crystallography",2,2,2,2,2,2,2
"Acta crystallographica. Section F, Structural biology and crystallization communications",2,2,2,2,2,2,2
Acta dermato-venereologica,1,1,1,1,1,1,1
Acta diabetologica,1,1,1,1,1,1,1
Acta neuropathologica,3,2,3,3,3,3,3
Acta ophthalmologica,1,1,1,1,1,1,1


In [109]:
# Found the 5 most common journal titles
dfgrouped.sort_values(by=['Article title'], ascending = False).head(10)

,Unnamed: 0,PMID/PMCID,Publisher,Journal title,Article title,Cost,Cleaned
NewTitle,,,,,,,
The Journal of biological chemistry,62,61,62,62,62,62,62
Nucleic acids research,29,28,29,29,29,29,29
PLoS neglected tropical diseases,23,23,23,23,23,23,23
PLoS pathogens,22,22,22,22,22,22,22
PLoS genetics,22,22,22,22,22,22,22
Human molecular genetics,19,17,19,19,19,19,19
Nature communications,18,18,18,18,18,18,18
The FEBS journal,18,16,18,18,18,18,18
The Biochemical journal,16,15,16,16,16,16,16


In [224]:
plosonedf = dfcleancost[dfcleancost['Journal title'] == 'PloS one']
biochemdf = dfcleancost[dfcleancost['Journal title'] == 'The Journal of biological chemistry']
neuleicdf = dfcleancost[dfcleancost['Journal title'] == 'Nucleic acids research']
neurodf = dfcleancost[dfcleancost['Journal title'] == 'NeuroImage']
tropdf = dfcleancost[dfcleancost['Journal title'] == 'PLoS neglected tropical diseases']

In [225]:
print('*****************************\nStatistics for PloS one\n*****************************')
print('Mean is: ' + str(plosonedf['Cost'].mean()))
print('Median is: ' + str(plosonedf['Cost'].median()))
print('Standard Deviation is: ' + str(plosonedf['Cost'].std()))

print('\n\n')

print('*****************************\nStatistics for The Journal of biological chemistry\n*****************************')
print('Mean is: ' + str(biochemdf['Cost'].mean()))
print('Median is: ' + str(biochemdf['Cost'].median()))
print('Standard Deviation is: ' + str(biochemdf['Cost'].std()))

print('\n\n')

print('*****************************\nStatistics for Nucleic acids research\n*****************************')
print('Mean is: ' + str(neuleicdf['Cost'].mean()))
print('Median is: ' + str(neuleicdf['Cost'].median()))
print('Standard Deviation is: ' + str(neuleicdf['Cost'].std()))

print('\n\n')

print('*****************************\nStatistics for NeuroImage\n*****************************')
print('Mean is: ' + str(neurodf['Cost'].mean()))
print('Median is: ' + str(neurodf['Cost'].median()))
print('Standard Deviation is: ' + str(neurodf['Cost'].std()))

print('\n\n')

print('*****************************\nPLoS neglected tropical diseases\n*****************************')
print('Mean is: ' + str(tropdf['Cost'].mean()))
print('Median is: ' + str(tropdf['Cost'].median()))
print('Standard Deviation is: ' + str(tropdf['Cost'].std()))

print('\n\n')


*****************************
Statistics for PloS one
*****************************
Mean is: 934.9821212121199
Median is: 896.975
Standard Deviation is: 194.96622876260946



*****************************
Statistics for The Journal of biological chemistry
*****************************
Mean is: 1412.5316923076925
Median is: 1324.57
Standard Deviation is: 376.6101872940426



*****************************
Statistics for Nucleic acids research
*****************************
Mean is: 1162.344827586207
Median is: 852.0
Standard Deviation is: 442.15093381769765



*****************************
Statistics for NeuroImage
*****************************
Mean is: 2212.1812903225805
Median is: 2326.43
Standard Deviation is: 273.19324434963534



*****************************
PLoS neglected tropical diseases
*****************************
Mean is: 1740.9239130434783
Median is: 1561.3
Standard Deviation is: 604.3845238808811



